# Replication of Japan Government Bond Yields

## Module Imports

In [49]:
import numpy as np
import itertools
import pandas as pd
import requests
import pandas_market_calendars as mcal
from scipy.interpolate import CubicSpline

## Setting dates

In [171]:
def get_next_biz_day(y, m, d):

    tse_calendar = mcal.get_calendar('JPX')
    start_date = pd.Timestamp(year=y, month=m, day=d)
    trading_days = tse_calendar.valid_days(start_date=start_date, end_date=start_date + pd.DateOffset(days=10))
    next_day = trading_days[
        trading_days.values.astype("datetime64[D]") > start_date.to_datetime64().astype("datetime64[D]")].min()
    return next_day.year, next_day.month, next_day.day

base_date = (2023, 12, 26)
base_date_ts = pd.Timestamp("-".join(str(v) for v in base_date))
ref_date = get_next_biz_day(*base_date)
ref_date_ts = pd.Timestamp("-".join(str(v) for v in ref_date))

base_date, ref_date

((2023, 12, 26), (2023, 12, 27))

## Data download

In [172]:
def download_csv(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        return True
    else:
        return False


def download_jsda_data(date):
    
    y, m, d = date
    filename = f"S{(y % 100) * 10000 + m * 100 + d:06d}.csv"
    url = f"https://market.jsda.or.jp/shijyo/saiken/baibai/baisanchi/files/{str(y)}/{filename}"
    
    if download_csv(url, filename):
        return filename
    else:
        raise RuntimeError(f"{filename} not found at https://market.jsda.or.jp/")
    

def download_mof_data():
    filename = "jgbcme.csv"
    url = f"https://www.mof.go.jp/english/policy/jgbs/reference/interest_rate/{filename}"
    download_csv(url, filename)
    
jsda_file = download_jsda_data(ref_date)
download_mof_data()

## Processing JSDA data

In [173]:
data = pd.read_csv(jsda_file, encoding="cp932", header=None)
header = pd.read_excel("dataheader.xlsx", header=None).iloc[:, 0]
data.columns = header
data = data[["IssueType", "Code", "DueDate", "CouponRate", "AverageCompoundYield"]]
data.head()

,IssueType,Code,DueDate,CouponRate,AverageCompoundYield
0,1,11850074,20240109,99.999,999.999
1,1,11670074,20240110,99.999,999.999
2,1,11870074,20240115,99.999,999.999
3,1,11330074,20240122,99.999,999.999
4,1,11880074,20240122,99.999,999.999


In [174]:
jgb_data = data[data["IssueType"] == 2].copy()
jgb_data["IssueNumber"] = np.floor(jgb_data["Code"]/10000).astype("int64")
jgb_data["Term"] = np.mod(jgb_data["Code"], 10000)
id_to_term = {
    42: 2,
    45: 5,
    67: 10,
    69: 20,
    68: 30,
    54: 40
}
jgb_terms = list(id_to_term.values())
jgb_data = jgb_data[jgb_data["Term"].isin(id_to_term)]
jgb_data["Term"] = jgb_data["Term"].replace(id_to_term)
jgb_data["DueDate"] = pd.to_datetime(jgb_data["DueDate"].astype(str), format="%Y%m%d")
jgb_data = jgb_data.sort_values(by="DueDate").set_index("Code")[
    ["IssueNumber", "Term", "DueDate", "CouponRate", "AverageCompoundYield"]]

jgb_data.head()

,IssueNumber,Term,DueDate,CouponRate,AverageCompoundYield
Code,,,,,
4320042,432,2,2024-01-01,0.005,0.005
4330042,433,2,2024-02-01,0.005,-0.192
4340042,434,2,2024-03-01,0.005,-0.105
690069,69,20,2024-03-20,2.100,-0.174
3330067,333,10,2024-03-20,0.600,-0.172


## Defining Variables

In [175]:
jgb_terms = list(id_to_term.values())

current_by_term = {
    term: max(jgb_data[jgb_data["Term"] == term]["IssueNumber"]) for term in jgb_terms
}
current_by_term

{2: 456, 5: 163, 10: 372, 20: 186, 30: 80, 40: 16}

In [176]:
grid = pd.DataFrame(index=range(1, 41))

for s, j in zip(["_prev", "", "_next"], [-1, 0, 1]):
    grid["date" + s] = [ref_date_ts + pd.DateOffset(years=i+j) for i in grid.index]

selected_terms = []

prev_y = 0
for y in jgb_terms:
    selected_terms.extend([y] * (y - prev_y))
    prev_y = y

grid["jgb_term"] = selected_terms
grid.head()

,date_prev,date,date_next,jgb_term
1,2023-12-27,2024-12-27,2025-12-27,2
2,2024-12-27,2025-12-27,2026-12-27,2
3,2025-12-27,2026-12-27,2027-12-27,5
4,2026-12-27,2027-12-27,2028-12-27,5
5,2027-12-27,2028-12-27,2029-12-27,5


## Finding Issues

In [177]:
def find_issue(grid_id, is_inside):
    
    grid_term, date_prev, grid_date, date_next = (
        grid.loc[grid_id, col] for col in ["jgb_term", "date_prev", "date", "date_next"])

    issues = jgb_data[(jgb_data["Term"] == grid_term)]
    
    if is_inside:
        issues = issues[(issues["DueDate"] > date_prev) & (issues["DueDate"] <= grid_date)]
    else:
        issues = issues[(issues["DueDate"] > grid_date) & (issues["DueDate"] < date_next)]

    if is_inside and (is_current := issues["IssueNumber"]==current_by_term[grid_term]).any():
        issues = issues[is_current]
        assert len(issues) == 1
        return issues.iloc[0].name

    else:
        if is_inside:
            min_diff = (grid_date - issues["DueDate"]).min()
            issues = issues[(grid_date - issues["DueDate"]) == min_diff]
        else:
            min_diff = (issues["DueDate"] - grid_date).min()
            issues = issues[(issues["DueDate"] - grid_date) == min_diff]
        
        if len(issues) == 1:
            return issues.iloc[0].name
        else:
            max_coupon = issues["CouponRate"].max()
            issues = issues[issues["CouponRate"] == max_coupon]
            if len(issues) == 1:
                return issues.iloc[0].name
            else:
                max_issue_no = issues["IssueNumber"].max()
                issues = issues[issues["IssueNumber"] == max_issue_no]
                if len(issues) == 1:
                    return issues.iloc[0].name
                elif len(issues) == 0:
                    return None
                else:
                    raise ValueError(f"Multiple canidates found for Grid:{grid_id}")

issue_ids = {}
for grid_id, is_inside in itertools.product(range(1, 41), [True, False]):
    if issue := find_issue(grid_id, is_inside):
        issue_ids.setdefault(grid_id, []).append(issue)


flatten = list(dict.fromkeys(v for l in issue_ids.values() for v in l))
tangent_points = jgb_data.loc[flatten]
tangent_points.head()

,IssueNumber,Term,DueDate,CouponRate,AverageCompoundYield
Code,,,,,
4430042,443,2,2024-12-01,0.005,-0.048
4440042,444,2,2025-01-01,0.005,-0.034
4550042,455,2,2025-12-01,0.005,0.041
4560042,456,2,2026-01-01,0.100,0.069
1500045,150,5,2026-12-20,0.005,0.075


## Interpolation by Cubic Spline

In [178]:
x = tangent_points["DueDate"].values.astype("datetime64[D]")
y = tangent_points["AverageCompoundYield"].values
cs = CubicSpline(x, y)

In [179]:
maturity_years = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 40]
maturities = [(ref_date_ts + pd.DateOffset(years=i)).to_datetime64().astype("datetime64[D]") for i in maturity_years]

yields = np.around(cs(maturities), decimals=3)
yields

array([-0.036,  0.065,  0.075,  0.158,  0.261,  0.289,  0.382,  0.493,
        0.578,  0.656,  1.086,  1.396,  1.546,  1.626,  1.742])

##  Compariosn with public data

In [180]:
import pandas as pd

def find_footer(file_path):
    length = 0
    found = 0
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            # Find first comma only line(",,,,,,,,,")
            if not found and not any(line.strip().split(",")): 
                found = i
            length += 1
            
    return length - found if found else 0

footer = find_footer('jgbcme.csv')

mof_data = pd.read_csv('jgbcme.csv', header=None, skiprows=2, skipfooter=footer, engine='python')
mof_data[0] = pd.to_datetime(mof_data[0], yearfirst=True)
mof_data = mof_data.set_index(0)
mof_data.columns = maturity_years
mof_data.head()

,1,2,3,4,5,6,7,8,9,10,15,20,25,30,40
0,,,,,,,,,,,,,,,
2023-12-01,-0.055,0.040,0.075,0.170,0.281,0.339,0.444,0.553,0.636,0.720,1.133,1.449,1.602,1.683,1.810
2023-12-04,-0.055,0.031,0.063,0.156,0.271,0.319,0.425,0.536,0.621,0.712,1.139,1.454,1.606,1.688,1.816
2023-12-05,-0.046,0.036,0.058,0.151,0.271,0.305,0.407,0.519,0.605,0.687,1.126,1.446,1.598,1.683,1.807
2023-12-06,-0.055,0.036,0.055,0.147,0.260,0.293,0.392,0.504,0.585,0.667,1.083,1.397,1.543,1.626,1.748
2023-12-07,-0.024,0.087,0.121,0.223,0.356,0.392,0.490,0.606,0.687,0.771,1.185,1.500,1.631,1.709,1.816


In [181]:
comparison = mof_data.loc[base_date_ts]
comparison.name = "Released"
comparison = comparison.to_frame() 
comparison["Replicated"] = yields
comparison
comparison["Diff"] = comparison["Replicated"] - comparison["Released"]
comparison

,Released,Replicated,Diff
1,-0.037,-0.036,0.001
2,0.065,0.065,0.000
3,0.076,0.075,-0.001
4,0.158,0.158,0.000
5,0.262,0.261,-0.001
6,0.290,0.289,-0.001
7,0.382,0.382,0.000
8,0.494,0.493,-0.001
9,0.578,0.578,0.000
10,0.657,0.656,-0.001
